In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [5]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("nyc-taxi-experiment")


2025/12/07 16:13:10 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1765120390682, experiment_id='1', last_update_time=1765120390682, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [8]:
# read the data and treat it
# change compression if needed

def read_dataframe(filename, compression='gzip'):
    df = pd.read_csv(filename, compression=compression)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)] 

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [9]:
df_train = read_dataframe('./data/green_tripdata_2021-01.csv.gz')
df_val = read_dataframe('./data/green_tripdata_2021-02.csv.gz')

C:\Users\seunj\AppData\Local\Temp\ipykernel_89460\1188441164.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, compression=compression)


In [10]:

len(df_train), len(df_val)

(73908, 61921)

In [11]:

# We will use feature engineering to create a new feature called PU_DO 
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:

# set the features we will use
categorical = ['PU_DO'] 
numerical = ['trip_distance']

# We will use the DictVectorizer to convert categorical variables into a format that can be used by the model
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:

# set the target variable

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:

# We will use the LinearRegression model to train the data
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715204860739

In [15]:
# save the model
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [16]:
# save the model with mlflow
with mlflow.start_run():

    mlflow.set_tag("developer", "jonah")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

🏃 View run debonair-lark-813 at: http://127.0.0.1:5000/#/experiments/1/runs/942a0fae0ded4aeda5f6a6a4592aefe0
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


## 2.0 Using xgb and Hyperopt

In [17]:
import xgboost as xgb

In [18]:

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

d:\MLOPS\mlops-zoomcamp-course\venv\Lib\site-packages\hyperopt\atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [19]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [20]:
# first we need to define the objective function
def objective(params):
    # the parameters are passed both to mlflow and to xgboost
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)

        # configure the parameters of the model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

        # once the model is trained, we can save it and make predictions
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)

        mlflow.log_metric("rmse", rmse)
        
    # we return the loss, which is the value we want to minimize
    # and the status of the optimization
    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
# define the search space

# note: when we use hp.loguniform, we are exploring the range exponention(low value, high value)

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),  # here we need to convert to int, as this value only takes int
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

# we can use the fmin function to minimize the objective function
best_result = fmin(
    fn=objective, # the function we defined above
    space=search_space, # the search space we defined above
    algo=tpe.suggest, # the algorithm we will use to search the space
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:11.82279                          
[1]	validation-rmse:11.45805                          
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

d:\MLOPS\mlops-zoomcamp-course\venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [17:22:42] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[2]	validation-rmse:11.11675                          
[3]	validation-rmse:10.79820                          
[4]	validation-rmse:10.50087                          
[5]	validation-rmse:10.22356                          
[6]	validation-rmse:9.96582                           
[7]	validation-rmse:9.72648                           
[8]	validation-rmse:9.50356                           
[9]	validation-rmse:9.29693                           
[10]	validation-rmse:9.10458                          
[11]	validation-rmse:8.92648                          
[12]	validation-rmse:8.76192                          
[13]	validation-rmse:8.60931                          
[14]	validation-rmse:8.46836                          
[15]	validation-rmse:8.33840                          
[16]	validation-rmse:8.21832                          
[17]	validation-rmse:8.10724                          
[18]	validation-rmse:8.00493                          
[19]	validation-rmse:7.91035                          
[20]	valid

d:\MLOPS\mlops-zoomcamp-course\venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [17:23:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:10.02745                                                   
[1]	validation-rmse:8.63317                                                    
[2]	validation-rmse:7.76461                                                    
[3]	validation-rmse:7.24561                                                    
[4]	validation-rmse:6.93366                                                    
[5]	validation-rmse:6.75062                                                    
[6]	validation-rmse:6.63870                                                    
[7]	validation-rmse:6.57014                                                    
[8]	validation-rmse:6.51980                                                    
[9]	validation-rmse:6.48813                                                    
[10]	validation-rmse:6.46729                                                   
[11]	validation-rmse:6.45251                                                   
[12]	validation-rmse:6.44170            

...

[0]	validation-rmse:11.77150                                                       
[1]	validation-rmse:11.36030                                                       
[2]	validation-rmse:10.97828                                                       
[3]	validation-rmse:10.62350                                                       
[4]	validation-rmse:10.29429                                                       
[5]	validation-rmse:9.98915                                                        
[6]	validation-rmse:9.70666                                                        
[7]	validation-rmse:9.44568                                                        
[8]	validation-rmse:9.20439                                                        
[9]	validation-rmse:8.98230                                                        
[10]	validation-rmse:8.77744                                                       
[11]	validation-rmse:8.58885                                                

d:\MLOPS\mlops-zoomcamp-course\venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [19:10:38] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:11.64454                                                       
[1]	validation-rmse:11.12689                                                       
[2]	validation-rmse:10.65654                                                       
[3]	validation-rmse:10.22962                                                       
[4]	validation-rmse:9.84280                                                        
[5]	validation-rmse:9.49366                                                        
[6]	validation-rmse:9.17747                                                        
[7]	validation-rmse:8.89380                                                        
[8]	validation-rmse:8.63703                                                        
[9]	validation-rmse:8.40788                                                        
[10]	validation-rmse:8.20410                                                       
[11]	validation-rmse:8.01964                                                

d:\MLOPS\mlops-zoomcamp-course\venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [19:14:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:10.94757                                                       
[1]	validation-rmse:9.94489                                                        
[2]	validation-rmse:9.15563                                                        
[3]	validation-rmse:8.54092                                                        
[4]	validation-rmse:8.06456                                                        
[5]	validation-rmse:7.70289                                                        
[6]	validation-rmse:7.42869                                                        
[7]	validation-rmse:7.21799                                                        
[8]	validation-rmse:7.05893                                                        
[9]	validation-rmse:6.93570                                                        
[10]	validation-rmse:6.84309                                                       
[11]	validation-rmse:6.77132                                                

In [25]:
mlflow.xgboost.autolog(disable=True)

In [26]:
with mlflow.start_run():
    best_params = {
        'learning_rate': 0.9531914478373814,
        'max_depth': 61,
        'min_child_weight': 0.4463857890057663,
        'objective': 'reg:linear',
        'reg_alpha': 0.006949308354101747,
        'reg_lambda': 0.20107023044259642,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    # once the model is trained, we can save it and make predictions
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")









d:\MLOPS\mlops-zoomcamp-course\venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [00:29:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:8.76476
[1]	validation-rmse:7.37101
[2]	validation-rmse:6.84340
[3]	validation-rmse:6.63616
[4]	validation-rmse:6.54621
[5]	validation-rmse:6.50627
[6]	validation-rmse:6.47853
[7]	validation-rmse:6.46355
[8]	validation-rmse:6.45772
[9]	validation-rmse:6.45046
[10]	validation-rmse:6.44724
[11]	validation-rmse:6.44529
[12]	validation-rmse:6.43989
[13]	validation-rmse:6.43320
[14]	validation-rmse:6.42939
[15]	validation-rmse:6.42562
[16]	validation-rmse:6.42117
[17]	validation-rmse:6.41685
[18]	validation-rmse:6.41368
[19]	validation-rmse:6.40920
[20]	validation-rmse:6.40646
[21]	validation-rmse:6.40445
[22]	validation-rmse:6.40137
[23]	validation-rmse:6.39965
[24]	validation-rmse:6.39838
[25]	validation-rmse:6.39604
[26]	validation-rmse:6.39456
[27]	validation-rmse:6.39184
[28]	validation-rmse:6.38970
[29]	validation-rmse:6.38729
[30]	validation-rmse:6.38680
[31]	validation-rmse:6.38221
[32]	validation-rmse:6.38121
[33]	validation-rmse:6.37887
[34]	validation-rmse:6.3

2025/12/08 00:30:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run bustling-snail-173 at: http://127.0.0.1:5000/#/experiments/1/runs/3f17e2d45c1a4fdf8db9aca8b993a0e6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [ ]:

# we pass the run id
logged_model = "runs:/0eeed647ce134080b75df6742764b655/models_mlflow"

# load model as a PyFuncModel
loaded_model = mlflow.pyfunc.load_model(logged_model)

# predict on pandas DataFrame
import pandas as pd
#loaded_model.predict(pd.DataFrame({'', 'trip_distance': [1.0]}))  # you need to pass the data in the same format as the model was trained